In [ ]:
%env PATH=~/anaconda3/envs/tesmo/bin/:$PATH
%env PYTHONPATH=~/anaconda3/envs/tesmo/lib/python3.8/site-packages/:$PYTHONPATH:
from babel_tools import load_motion, load_pelvis_traj
import os
import sys
import numpy as np
from pathlib import Path
import json
from PIL import Image
import trimesh
from tqdm import tqdm
import pandas as pd
import random
import pickle as pkl
import time
#from p_tqdm import p_map, p_uimap
import glob
import matplotlib.pyplot as plt
import pickle as pkl
import cv2

In [2]:
index_path = './dataset/HumanML3D/train_walk_ori_amass_path.txt'
filter_sequence = pd.read_csv(index_path, header=None, sep=',')    

In [3]:
files = [filter_sequence.loc[idx] for idx in range(len(filter_sequence))]
amass_dir = './amass_data'
ones = []
for one in tqdm(files):
    npz_file = os.path.join(amass_dir, one[1].replace(' ./pose_data/', '').replace('npy', 'npz')) # remove the space
    start_f = one[2]
    end_f = one[3]
    if os.path.exists(npz_file):
        ones.append(one)

  0%|          | 0/4751 [00:00<?, ?it/s]

100%|██████████| 4751/4751 [07:35<00:00, 10.44it/s]


In [4]:
ones = [one for one in ones if one[3]-one[2]>160]

In [5]:
orients_traj, trans_traj = [], []
orients_traf, trans_traf, velos_traf = [], [], []
clip_window = 20
fps = 20

def cal_velo(trans):
    dis_tot = 0.
    for i in range(len(trans)-1):
        dis_tot += np.linalg.norm(trans[i+1]-trans[i])
    return dis_tot / clip_window * fps

for one in tqdm(ones):
    npz_file = os.path.join(amass_dir, one[1].replace(' ./pose_data/', '').replace('npy', 'npz')) # remove the space
    start_f, end_f = one[2], one[3]
    orients, trans = load_pelvis_traj(npz_file, start_f, end_f)
    orients_traj.append(orients)
    trans_traj.append(trans)
    n_len = len(orients)
    for i in range(0, n_len-1, clip_window):
        ei = min(i + clip_window, n_len-1)
        si = min(ei-clip_window, i)
        if ei - si != clip_window:
            continue
        orients_clip.append(orients[si:ei+1])
        trans_clip.append(trans[si:ei+1])
        velos_clip.append(cal_velo(trans_clip[-1]))

100%|██████████| 2228/2228 [11:25<00:00,  3.25it/s]


In [44]:
def cal_velo(trans):
    dis_tot = np.linalg.norm(trans[1:]-trans[:-1], axis=-1).sum()
    return dis_tot / clip_window * fps
trans_clip_2d = []
orients_clip_2d = []
rot_clip_2d = []
velos_clip_2d = []
rots_clip_2d = []
dir_clip_2d = []
dis_clip_2d = []
for trans, orient in tqdm(zip(trans_clip, orients_clip)):
    trans_clip_2d.append(trans[:, :2])
    rot_vecs = np.array([np.array([[0.,0.,1.]]) @ cv2.Rodrigues(o)[0].T for o in orient])[:, 0, :2]
    orients_clip_2d.append(rot_vecs[:, :2])
    rots_clip_2d.append(np.arctan2(rot_vecs[:, 1], rot_vecs[:, 0]))
    move_vec = trans[1:, :2] - trans[:-1, :2]
    dir_clip_2d.append(np.arctan2(move_vec[:, 1], move_vec[:, 0]))
    velos_clip_2d.append(cal_velo(trans_clip_2d[-1]))
    dis_clip_2d.append(np.linalg.norm(trans[1:]-trans[:-1], axis=-1).cumsum())
trans_traj_2d = []
orients_traj_2d = []
rot_traj_2d = []
velos_traj_2d = []
rots_traj_2d = []
dir_traj_2d = []
dis_traj_2d = []
for trans, orient in tqdm(zip(trans_traj, orients_traj)):
    trans_traj_2d.append(trans[:, :2])
    rot_vecs = np.array([np.array([[0.,0.,1.]]) @ cv2.Rodrigues(o)[0].T for o in orient])[:, 0, :2]
    orients_traj_2d.append(rot_vecs[:, :2])
    rots_traj_2d.append(np.arctan2(rot_vecs[:, 1], rot_vecs[:, 0]))
    move_vec = trans[1:, :2] - trans[:-1, :2]
    dir_traj_2d.append(np.arctan2(move_vec[:, 1], move_vec[:, 0]))
    velos_traj_2d.append(cal_velo(trans[-1]))
    dis_traj_2d.append(np.linalg.norm(trans[1:]-trans[:-1], axis=-1).cumsum())

11112it [00:02, 4546.08it/s]
2228it [00:02, 948.61it/s]


In [ ]:
dir_clip_2d = np.stack(dir_clip_2d)
rots_clip_2d = np.stack(rots_clip_2d)
dis_clip_2d = np.stack(rots_clip_2d)
angle_diff = (dir_clip_2d - rots_clip_2d[:, 1:]) % (np.pi * 2)
angle_diff = np.where(angle_diff > np.pi, np.pi * 2 - angle_diff, angle_diff)
velos_clip_2d = np.stack(velos_clip_2d)
(velos_clip_2d > 0.5).sum()
mask = ((angle_diff.max(axis=-1) < 30/180 * np.pi) & (velos_clip_2d > 0.1))
with open('../files/motion_clips_db_2s.pkl', 'wb') as f:
    pkl.dump(dict(orients_traj=orients_traj, trans_traj=trans_traj, orients_clip=orients_clip, trans_clip=trans_clip, velos_clip=velos_clip, dis_clip_2d=dis_clip_2d), f)

In [55]:
with open('../files/motion_clips_db_2d_2s_filtered.pkl', 'wb') as f:
    pkl.dump(dict(trans_clip_2d=np.stack(trans_clip_2d)[mask], orients_clip_2d=np.stack(orients_clip_2d)[mask], velos_clip_2d=np.stack(velos_clip_2d)[mask], rots_clip_2d=np.stack(rots_clip_2d)[mask], dis_clip_2d=np.stack(dis_clip_2d)[mask]), f)

In [ ]:
with open('../files/motion_clips_db_2s.pkl', 'rb') as f:
    data = pkl.load(f)
plt.hist(data['velos_clip'], bins=100)